### LSTM in PyTorch for Character Generation

#on top of the code by LeanManager for my personal understanding. Original copyright for LSTM belongs to Hochreiter & Schmidhuber. 

In [74]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F


In [75]:
# open text file and read in data as `text`

#with open('anna.txt', 'r') as f:
with open('anna.txt', 'r') as f:

    text = f.read()


'''set("some_string")'''
#creates a dictionary of unique values from the items of the iterables passed to it.
#text = "Aashish"
#print(set(text)) prints {'s', 'A', 'h', 'i'}

#print(set(text))

'''print(tuple(set(text)))'''
#converts the dictionary into a comma-separated tuple



'print(tuple(set(text)))'

In [76]:
# encode the text and map each character to an integer and vice versa

# we create two dictonaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers



character_vocabulary = tuple(set(text))

int2char = dict(enumerate(character_vocabulary))
# creates {0: ')', 1: 'y', 2: 'a', 3: '/', 4: 'B', 5: 'f', 6: 'U', 7: 'X', .......and so on }

#just reverse the above key-value pairs 
char2int = {ch: ii for ii, ch in int2char.items()}

#convert the whole text into the corresponding integer value
each_char_in_encodeed_integer_form = np.array([char2int[ch] for ch in text])


In [77]:
def one_hot_encoder(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

## Making training mini-batches


To train on this data, we also want to create mini-batches for training. Remember that we want our batches to be multiple sequences of some desired number of sequence steps. Considering a simple example, our batches would look like this:

<img src="assets/sequence_batching@1x.png" width=500px>


<br>

In this example, we'll take the encoded characters (passed in as the `arr` parameter) and split them into multiple sequences, given by `no_of_seqs_per_batch` (also refered to as "batch size" in other places). Each of those sequences will be `no_of_timesteps_per_sequence` long.

### Creating Batches

**1. The first thing we need to do is discard some of the text so we only have completely full batches. **

Each batch contains $N \times M$ characters, where $N$ is the batch size (the number of sequences) and $M$ is the number of steps. Then, to get the total number of batches, $K$, we can make from the array `arr`, you divide the length of `arr` by the number of characters per batch. Once you know the number of batches, you can get the total number of characters to keep from `arr`, $N * M * K$.

**2. After that, we need to split `arr` into $N$ sequences. ** 

You can do this using `arr.reshape(size)` where `size` is a tuple containing the dimensions sizes of the reshaped array. We know we want $N$ sequences, so let's make that the size of the first dimension. For the second dimension, you can use `-1` as a placeholder in the size, it'll fill up the array with the appropriate data for you. After this, you should have an array that is $N \times (M * K)$.

**3. Now that we have this array, we can iterate through it to get our batches. **

The idea is each batch is a $N \times M$ window on the $N \times (M * K)$ array. For each subsequent batch, the window moves over by `no_of_timesteps_per_sequence`. We also want to create both the input and target arrays. Remember that the targets are the inputs shifted over one character. The way I like to do this window is use `range` to take steps of size `no_of_timesteps_per_sequence` from $0$ to `arr.shape[1]`, the total number of steps in each sequence. That way, the integers you get from `range` always point to the start of a batch, and each window is `no_of_timesteps_per_sequence` wide.

In [78]:

#This is a generator function that returns batches one by one.
def get_batches(array_of_characters, no_of_seqs_per_batch, no_of_timesteps_per_sequence):# _, 10, 50
    '''Create a generator that returns batches of size
       no_of_seqs_per_batch x no_of_timesteps_per_sequence from array_of_characters.
       
       Arguments
       ---------
       array_of_characters: Array you want to make batches from
       no_of_seqs_per_batch: Batch size, the number of sequences per batch
       no_of_timesteps_per_sequence: Number of sequence steps per batch
    '''
    
    
    batch_size = no_of_seqs_per_batch * no_of_timesteps_per_sequence
    no_of_batches = len(array_of_characters)//batch_size
    
    # Keep only enough characters to make full batches
    array_of_characters = array_of_characters[:no_of_batches * batch_size]
    
    # Reshape into no_of_seqs_per_batch rows (In our example, 2)
    array_of_characters = array_of_characters.reshape((no_of_seqs_per_batch, -1))
       
    for n in range(0, array_of_characters.shape[1], no_of_timesteps_per_sequence): #simple way to get the batches
        
        # The features
        x = array_of_characters[:, n:n+no_of_timesteps_per_sequence]
        
        # The targets, shifted by one
        y = np.zeros_like(x)
        
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], array_of_characters[:, n+no_of_timesteps_per_sequence] #sweet
#             print(x)
#             print(y)
            
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], array_of_characters[:, 0]
        yield x, y

In [79]:
class LSTM_Char_Generation(nn.Module):
    
    
    def __init__(self, unique_vocabulary_tokens, no_of_timesteps_per_sequence=100, no_of_hidden_nodes_in_each_lstm_cell=256, no_of_stacked_lstm_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.no_of_stacked_lstm_layers = no_of_stacked_lstm_layers
        self.no_of_hidden_nodes_in_each_lstm_cell = no_of_hidden_nodes_in_each_lstm_cell
        self.lr = lr
        
        # Creating character dictionaries
        self.character_vocabulary = unique_vocabulary_tokens
        
        self.int2char = dict(enumerate(self.character_vocabulary))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## Define the LSTM
        self.lstm = nn.LSTM(len(self.character_vocabulary), no_of_hidden_nodes_in_each_lstm_cell, no_of_stacked_lstm_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## Define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## Define the final, fully-connected output layer
        self.fc = nn.Linear(no_of_hidden_nodes_in_each_lstm_cell, len(self.character_vocabulary))
        #output the logit for each item
        
        # Initialize the weights of the fully-connected layer
        self.init_weights()
      
    
    def forward(self, x, hidden_and_cell_states):
        import time
        ''' Forward pass through the network. 
            These inputs are x, and the hidden and cell states `hidden_and_cell_states`. '''
        
        #Get x, and the new hidden state (h, c) from the lstm
        x, (h, c) = self.lstm(x, hidden_and_cell_states)
        
        ## Pass x through the dropout layer
        x = self.dropout(x)#will have to be taken care using model.evaluate()
        
        #print(x.shape) torch.Size([128, 100, 512])
        #batch size, no of timesteps, no of hidden nodes        
        
        
        
        # Stack up LSTM outputs using view
        x = x.view(x.size()[0]*x.size()[1], self.no_of_hidden_nodes_in_each_lstm_cell)
        #print(x.shape) torch.Size([12800, 512])
        
        #This reshaping properly preserves each individual sample. An example can be found at the end of the file
        
        #12800 samples work as a batch
        
        #In case you want to use just the last timestep's output for time-series prediction, you do not have
        #to use the fully-connected layer.
        
        
        ## Put x through the fully-connected layer
        x = self.fc(x)
        
        # Return x and the hidden state (h, c)
        return x, (h, c)
    
    
    def predict(self, char, h=None, cuda=False, top_k=None):
        
        ''' Given a character, predict the next character.
        
            Returns the predicted character and the hidden state.
        '''
        
        #The output of our RNN is from a fully-connected layer and it outputs a distribution of next-character 
        #scores. To actually get the next character, we apply a softmax function, 
        #which gives us a probability distribution that we can then sample to predict the next character
        
        
        if cuda:
            self.cuda()
        else:
            self.cpu()
        
        if h is None:
            h = self.init_hidden(1) 
        
        x = np.array([[self.char2int[char]]])
        x = one_hot_encoder(x, len(self.character_vocabulary))
        
        inputs = torch.from_numpy(x)
        
        if cuda:
            inputs = inputs.cuda()
        
        h = tuple([each.data for each in h])
        out, h = self.forward(inputs, h)

        p = F.softmax(out, dim=1).data
        
        if cuda:
            p = p.cpu()
        
        if top_k is None:
            top_ch = np.arange(len(self.character_vocabulary))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        p = p.numpy().squeeze()
        
        char = np.random.choice(top_ch, p=p/p.sum())
            
        return self.int2char[char], h
    
    def init_weights(self):
        ''' Initialize weights for fully connected layer '''
        initrange = 0.1
        
        # Set bias tensor to all zeros
        self.fc.bias.data.fill_(0)
        # FC weights as random uniform
        self.fc.weight.data.uniform_(-1, 1)
        
    def init_hidden(self, no_of_seqs_per_batch):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x no_of_seqs_per_batch x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        return (weight.new(self.no_of_stacked_lstm_layers, no_of_seqs_per_batch, self.no_of_hidden_nodes_in_each_lstm_cell).zero_(),
                weight.new(self.no_of_stacked_lstm_layers, no_of_seqs_per_batch, self.no_of_hidden_nodes_in_each_lstm_cell).zero_())
        

### A note on the `predict`  function

The output of our RNN is from a fully-connected layer and it outputs a **distribution of next-character scores**.

To actually get the next character, we apply a softmax function, which gives us a *probability* distribution that we can then sample to predict the next character.

In [96]:
def train(network, data, epochs=10, no_of_seqs_per_batch=10, no_of_timesteps_per_sequence=50, lr=0.001, clip=5, val_frac=0.1, cuda=False, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        network: LSTM_Character_Generation network
        data: text data to train the network
        epochs: Number of epochs to train
        no_of_seqs_per_batch: Number of mini-sequences per mini-batch, aka batch size
        no_of_timesteps_per_sequence: Number of character steps in a sequence
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        cuda: Train with CUDA on a GPU
        print_every: Number of steps for printing training and validation loss
    
    '''
    
    network.train()
    
    optimizer = torch.optim.Adam(network.parameters(), lr=lr)
    
    criterion = nn.CrossEntropyLoss()#the suitable loss for such outputs with one 1 and rest 0 values
    
    # create training and validation data
    validation_index = int(len(data)*(1-val_frac))
    data, validation_data = data[:validation_index], data[validation_index:] #training and validation data separation
    
    if cuda:
        network.cuda()
    
    counter = 0
    no_of_unique_characters_in_vocab = len(network.character_vocabulary)
    
    for e in range(epochs):
        
        #Note that we will initialize the hidden state at the beginning of each epoch of training
        h = network.init_hidden(no_of_seqs_per_batch)
        
        for x, y in get_batches(data, no_of_seqs_per_batch, no_of_timesteps_per_sequence):
            
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encoder(x, no_of_unique_characters_in_vocab)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if cuda:
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            network.zero_grad()
            
            output, h = network.forward(inputs, h)
            
            loss = criterion(output, targets.view(no_of_seqs_per_batch*no_of_timesteps_per_sequence).type(torch.cuda.LongTensor))

            loss.backward()
            
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(network.parameters(), clip)

            optimizer.step()
            
            if counter % print_every == 0:
                
                # Get validation loss
                validation_h = network.init_hidden(no_of_seqs_per_batch)
                validation_losses = []
                
                for x, y in get_batches(validation_data, no_of_seqs_per_batch, no_of_timesteps_per_sequence):
                    
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encoder(x, no_of_unique_characters_in_vocab)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    validation_h = tuple([each.data for each in validation_h])
                    
                    inputs, targets = x, y
                    if cuda:
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, validation_h = network.forward(inputs, validation_h)
                    validation_loss = criterion(output, targets.view(no_of_seqs_per_batch*no_of_timesteps_per_sequence).type(torch.cuda.LongTensor))
                
                    validation_losses.append(validation_loss.item())
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Training Loss: {:.4f}...".format(loss.item()),
                      "Validation Loss: {:.4f}".format(np.mean(validation_losses)))

In [97]:

if 'network' in locals(): # function returns a dictionary containing the variables defined in the local namespace
    del network

In [98]:
# Initialize the network

network = LSTM_Char_Generation(character_vocabulary, no_of_hidden_nodes_in_each_lstm_cell=512, no_of_stacked_lstm_layers=2)
print(network)

LSTM_Char_Generation(
  (lstm): LSTM(83, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5)
  (fc): Linear(in_features=512, out_features=83, bias=True)
)


In [99]:
no_of_seqs_per_batch, no_of_timesteps_per_sequence = 128, 100
#set to training mode
network.train()
train(network, each_char_in_encodeed_integer_form, epochs= 60, no_of_seqs_per_batch=no_of_seqs_per_batch, no_of_timesteps_per_sequence=no_of_timesteps_per_sequence, lr=0.0001, cuda=True, print_every=10)

Epoch: 1/60... Step: 10... Training Loss: 3.8262... Validation Loss: 3.7819
Epoch: 1/60... Step: 20... Training Loss: 3.4719... Validation Loss: 3.4710
Epoch: 1/60... Step: 30... Training Loss: 3.3895... Validation Loss: 3.3783
Epoch: 1/60... Step: 40... Training Loss: 3.3129... Validation Loss: 3.3285
Epoch: 1/60... Step: 50... Training Loss: 3.3073... Validation Loss: 3.3019
Epoch: 1/60... Step: 60... Training Loss: 3.2811... Validation Loss: 3.2815
Epoch: 1/60... Step: 70... Training Loss: 3.2524... Validation Loss: 3.2675
Epoch: 1/60... Step: 80... Training Loss: 3.2547... Validation Loss: 3.2503
Epoch: 1/60... Step: 90... Training Loss: 3.2494... Validation Loss: 3.2376
Epoch: 1/60... Step: 100... Training Loss: 3.2260... Validation Loss: 3.2215
Epoch: 1/60... Step: 110... Training Loss: 3.2123... Validation Loss: 3.2042
Epoch: 1/60... Step: 120... Training Loss: 3.1753... Validation Loss: 3.1862
Epoch: 1/60... Step: 130... Training Loss: 3.1648... Validation Loss: 3.1694
Epoch: 2

Epoch: 8/60... Step: 1080... Training Loss: 2.2618... Validation Loss: 2.3175
Epoch: 8/60... Step: 1090... Training Loss: 2.2234... Validation Loss: 2.3127
Epoch: 8/60... Step: 1100... Training Loss: 2.2271... Validation Loss: 2.3113
Epoch: 8/60... Step: 1110... Training Loss: 2.2376... Validation Loss: 2.3087
Epoch: 9/60... Step: 1120... Training Loss: 2.2434... Validation Loss: 2.3036
Epoch: 9/60... Step: 1130... Training Loss: 2.2310... Validation Loss: 2.3003
Epoch: 9/60... Step: 1140... Training Loss: 2.2444... Validation Loss: 2.2987
Epoch: 9/60... Step: 1150... Training Loss: 2.2336... Validation Loss: 2.2911
Epoch: 9/60... Step: 1160... Training Loss: 2.1945... Validation Loss: 2.2925
Epoch: 9/60... Step: 1170... Training Loss: 2.2019... Validation Loss: 2.2879
Epoch: 9/60... Step: 1180... Training Loss: 2.2161... Validation Loss: 2.2868
Epoch: 9/60... Step: 1190... Training Loss: 2.2408... Validation Loss: 2.2813
Epoch: 9/60... Step: 1200... Training Loss: 2.1881... Validation

Epoch: 16/60... Step: 2120... Training Loss: 2.0039... Validation Loss: 2.0945
Epoch: 16/60... Step: 2130... Training Loss: 1.9833... Validation Loss: 2.0946
Epoch: 16/60... Step: 2140... Training Loss: 1.9669... Validation Loss: 2.0981
Epoch: 16/60... Step: 2150... Training Loss: 2.0079... Validation Loss: 2.0905
Epoch: 16/60... Step: 2160... Training Loss: 1.9978... Validation Loss: 2.0998
Epoch: 16/60... Step: 2170... Training Loss: 1.9851... Validation Loss: 2.0929
Epoch: 16/60... Step: 2180... Training Loss: 1.9923... Validation Loss: 2.0957
Epoch: 16/60... Step: 2190... Training Loss: 1.9778... Validation Loss: 2.1082
Epoch: 16/60... Step: 2200... Training Loss: 1.9868... Validation Loss: 2.0972
Epoch: 16/60... Step: 2210... Training Loss: 1.9548... Validation Loss: 2.0922
Epoch: 16/60... Step: 2220... Training Loss: 2.0089... Validation Loss: 2.0890
Epoch: 17/60... Step: 2230... Training Loss: 1.9645... Validation Loss: 2.0803
Epoch: 17/60... Step: 2240... Training Loss: 1.9681.

Epoch: 23/60... Step: 3160... Training Loss: 1.8537... Validation Loss: 2.0178
Epoch: 23/60... Step: 3170... Training Loss: 1.8384... Validation Loss: 1.9549
Epoch: 23/60... Step: 3180... Training Loss: 1.8131... Validation Loss: 1.9500
Epoch: 23/60... Step: 3190... Training Loss: 1.8430... Validation Loss: 1.9621
Epoch: 24/60... Step: 3200... Training Loss: 1.8583... Validation Loss: 1.9599
Epoch: 24/60... Step: 3210... Training Loss: 1.8589... Validation Loss: 1.9571
Epoch: 24/60... Step: 3220... Training Loss: 1.8334... Validation Loss: 1.9399
Epoch: 24/60... Step: 3230... Training Loss: 1.8090... Validation Loss: 1.9590
Epoch: 24/60... Step: 3240... Training Loss: 1.8006... Validation Loss: 1.9515
Epoch: 24/60... Step: 3250... Training Loss: 1.8550... Validation Loss: 1.9509
Epoch: 24/60... Step: 3260... Training Loss: 1.8430... Validation Loss: 1.9444
Epoch: 24/60... Step: 3270... Training Loss: 1.8531... Validation Loss: 1.9333
Epoch: 24/60... Step: 3280... Training Loss: 1.8459.

Epoch: 31/60... Step: 4200... Training Loss: 1.7635... Validation Loss: 1.8491
Epoch: 31/60... Step: 4210... Training Loss: 1.7336... Validation Loss: 1.8452
Epoch: 31/60... Step: 4220... Training Loss: 1.7704... Validation Loss: 1.8419
Epoch: 31/60... Step: 4230... Training Loss: 1.7024... Validation Loss: 1.8434
Epoch: 31/60... Step: 4240... Training Loss: 1.7207... Validation Loss: 1.8543
Epoch: 31/60... Step: 4250... Training Loss: 1.7100... Validation Loss: 1.8460
Epoch: 31/60... Step: 4260... Training Loss: 1.7330... Validation Loss: 1.8367
Epoch: 31/60... Step: 4270... Training Loss: 1.7231... Validation Loss: 1.8425
Epoch: 31/60... Step: 4280... Training Loss: 1.7188... Validation Loss: 1.8493
Epoch: 31/60... Step: 4290... Training Loss: 1.6959... Validation Loss: 1.8383
Epoch: 31/60... Step: 4300... Training Loss: 1.7386... Validation Loss: 1.8407
Epoch: 32/60... Step: 4310... Training Loss: 1.7802... Validation Loss: 1.8438
Epoch: 32/60... Step: 4320... Training Loss: 1.7401.

Epoch: 38/60... Step: 5240... Training Loss: 1.6492... Validation Loss: 1.7859
Epoch: 38/60... Step: 5250... Training Loss: 1.6563... Validation Loss: 1.7928
Epoch: 38/60... Step: 5260... Training Loss: 1.6248... Validation Loss: 1.7920
Epoch: 38/60... Step: 5270... Training Loss: 1.6344... Validation Loss: 1.7880
Epoch: 38/60... Step: 5280... Training Loss: 1.6439... Validation Loss: 1.7977
Epoch: 39/60... Step: 5290... Training Loss: 1.6602... Validation Loss: 1.7974
Epoch: 39/60... Step: 5300... Training Loss: 1.6404... Validation Loss: 1.7888
Epoch: 39/60... Step: 5310... Training Loss: 1.6618... Validation Loss: 1.7856
Epoch: 39/60... Step: 5320... Training Loss: 1.6669... Validation Loss: 1.8084
Epoch: 39/60... Step: 5330... Training Loss: 1.6185... Validation Loss: 1.7834
Epoch: 39/60... Step: 5340... Training Loss: 1.6439... Validation Loss: 1.7899
Epoch: 39/60... Step: 5350... Training Loss: 1.6429... Validation Loss: 1.7862
Epoch: 39/60... Step: 5360... Training Loss: 1.6663.

Epoch: 46/60... Step: 6280... Training Loss: 1.5699... Validation Loss: 1.7335
Epoch: 46/60... Step: 6290... Training Loss: 1.5956... Validation Loss: 1.7416
Epoch: 46/60... Step: 6300... Training Loss: 1.5593... Validation Loss: 1.7317
Epoch: 46/60... Step: 6310... Training Loss: 1.5653... Validation Loss: 1.7297
Epoch: 46/60... Step: 6320... Training Loss: 1.5918... Validation Loss: 1.7303
Epoch: 46/60... Step: 6330... Training Loss: 1.5769... Validation Loss: 1.7236
Epoch: 46/60... Step: 6340... Training Loss: 1.5697... Validation Loss: 1.7248
Epoch: 46/60... Step: 6350... Training Loss: 1.5527... Validation Loss: 1.7249
Epoch: 46/60... Step: 6360... Training Loss: 1.5792... Validation Loss: 1.7238
Epoch: 46/60... Step: 6370... Training Loss: 1.5658... Validation Loss: 1.7296
Epoch: 46/60... Step: 6380... Training Loss: 1.5399... Validation Loss: 1.7359
Epoch: 46/60... Step: 6390... Training Loss: 1.5977... Validation Loss: 1.7293
Epoch: 47/60... Step: 6400... Training Loss: 1.5449.

Epoch: 53/60... Step: 7320... Training Loss: 1.5275... Validation Loss: 1.6862
Epoch: 53/60... Step: 7330... Training Loss: 1.5431... Validation Loss: 1.6849
Epoch: 53/60... Step: 7340... Training Loss: 1.5217... Validation Loss: 1.6815
Epoch: 53/60... Step: 7350... Training Loss: 1.4974... Validation Loss: 1.6843
Epoch: 53/60... Step: 7360... Training Loss: 1.5193... Validation Loss: 1.6855
Epoch: 54/60... Step: 7370... Training Loss: 1.5357... Validation Loss: 1.6843
Epoch: 54/60... Step: 7380... Training Loss: 1.5302... Validation Loss: 1.6815
Epoch: 54/60... Step: 7390... Training Loss: 1.5383... Validation Loss: 1.6800
Epoch: 54/60... Step: 7400... Training Loss: 1.4941... Validation Loss: 1.6829
Epoch: 54/60... Step: 7410... Training Loss: 1.4664... Validation Loss: 1.6765
Epoch: 54/60... Step: 7420... Training Loss: 1.5464... Validation Loss: 1.6789
Epoch: 54/60... Step: 7430... Training Loss: 1.5359... Validation Loss: 1.6831
Epoch: 54/60... Step: 7440... Training Loss: 1.5350.

## Getting the best model

To set your hyperparameters to get the best performance, you'll want to watch the training and validation losses. If your training loss is much lower than the validation loss, you're overfitting. Increase regularization (more dropout) or use a smaller network. If the training and validation losses are close, you're underfitting so you can increase the size of the network.

## Hyperparameters

Here are the hyperparameters for the network.

In defining the model:
* `n_hidden` - The number of units in the hidden layers.
* `n_layers` - Number of hidden LSTM layers to use.

We assume that dropout probability and learning rate will be kept at the default, in this example.

And in training:
* `no_of_seqs_per_batch` - Number of sequences running through the network in one pass.
* `no_of_timesteps_per_sequence` - Number of characters in the sequence the network is trained on. Larger is better typically, the network will learn more long range dependencies. But it takes longer to train. 100 is typically a good number here.
* `lr` - Learning rate for training

Here's some good advice from Andrej Karpathy on training the network. I'm going to copy it in here for your benefit, but also link to [where it originally came from](https://github.com/karpathy/char-rnn#tips-and-tricks).

> ## Tips and Tricks

>### Monitoring Validation Loss vs. Training Loss
>If you're somewhat new to Machine Learning or Neural Networks it can take a bit of expertise to get good models. The most important quantity to keep track of is the difference between your training loss (printed during training) and the validation loss (printed once in a while when the RNN is run on the validation data (by default every 1000 iterations)). In particular:

> - If your training loss is much lower than validation loss then this means the network might be **overfitting**. Solutions to this are to decrease your network size, or to increase dropout. For example you could try dropout of 0.5 and so on.
> - If your training/validation loss are about equal then your model is **underfitting**. Increase the size of your model (either number of layers or the raw number of neurons per layer)

> ### Approximate number of parameters

> The two most important parameters that control the model are `n_hidden` and `n_layers`. I would advise that you always use `n_layers` of either 2/3. The `n_hidden` can be adjusted based on how much data you have. The two important quantities to keep track of here are:

> - The number of parameters in your model. This is printed when you start training.
> - The size of your dataset. 1MB file is approximately 1 million characters.

>These two should be about the same order of magnitude. It's a little tricky to tell. Here are some examples:

> - I have a 100MB dataset and I'm using the default parameter settings (which currently print 150K parameters). My data size is significantly larger (100 mil >> 0.15 mil), so I expect to heavily underfit. I am thinking I can comfortably afford to make `n_hidden` larger.
> - I have a 10MB dataset and running a 10 million parameter model. I'm slightly nervous and I'm carefully monitoring my validation loss. If it's larger than my training loss then I may want to try to increase dropout a bit and see if that helps the validation loss.

> ### Best models strategy

>The winning strategy to obtaining very good models (if you have the compute time) is to always err on making the network larger (as large as you're willing to wait for it to compute) and then try different dropout values (between 0,1). Whatever model has the best validation performance (the loss, written in the checkpoint filename, low is good) is the one you should use in the end.

>It is very common in deep learning to run many different models with many different hyperparameter settings, and in the end take whatever checkpoint gave the best validation performance.

>By the way, the size of your training and validation splits are also parameters. Make sure you have a decent amount of data in your validation set or otherwise the validation performance will be noisy and not very informative.

After training, we'll save the model so we can load it again later if we need too. Here I'm saving the parameters needed to create the same architecture, the hidden layer hyperparameters and the text characters.

In [100]:
# change the name, for saving multiple files
model_name = 'lstm_epoch.network'
checkpoint = {'no_of_hidden_nodes_in_each_lstm_cell': network.no_of_hidden_nodes_in_each_lstm_cell,
              'no_of_stacked_lstm_layers': network.no_of_stacked_lstm_layers,
              'state_dict': network.state_dict(),
              'tokens': network.character_vocabulary}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

## Sampling

Now that the model is trained, we'll want to sample from it. To sample, we pass in a character and have the network predict the next character. Then we take that character, pass it back in, and get another predicted character. Just keep doing this and you'll generate a bunch of text!

### Top K sampling

Our predictions come from a categorcial probability distribution over all the possible characters. We can make the sample text and make it more reasonable to handle (with less variables) by only considering some $K$ most probable characters. This will prevent the network from giving us completely absurd characters while allowing it to introduce some noise and randomness into the sampled text.

Typically you'll want to prime the network so you can build up a hidden state. Otherwise the network will start out generating characters at random. In general the first bunch of characters will be a little rough since it hasn't built up a long history of characters to predict from.

In [101]:
def sample(trained_network_which_will_sample, no_of_characters_to_generate_in_the_sequence, 
           initilaization_characters_instead_of_random_start='The', top_k=None, cuda=False):
        
    if cuda:
        trained_network_which_will_sample.cuda()
    else:
        trained_network_which_will_sample.cpu()

    #work on evaluation mode    
    trained_network_which_will_sample.eval()
    
    # First off, run through the prime/ non-random characters
    characters_list = [ch for ch in initilaization_characters_instead_of_random_start]
    
    #print(characters_list)
#     import time
#     time.sleep(333)
    
    #In RNN, you learn the weights for differnet nodes, however, you do not learn the states. Hence, even during
    #evaluation, you have to initialize the hidden and cell states.
    initialization_of_hidden_state = trained_network_which_will_sample.init_hidden(1)
    
    for character in initilaization_characters_instead_of_random_start:
        character, hidden_state = trained_network_which_will_sample.predict(character, initialization_of_hidden_state, cuda=cuda, top_k=top_k)

    characters_list.append(character)
    
    # Now pass in the previous character and get a new one
    for index in range(no_of_characters_to_generate_in_the_sequence):
        
        character, hidden_state = trained_network_which_will_sample.predict(characters_list[-1], hidden_state, cuda=cuda, top_k=top_k)
        characters_list.append(character)

    return ''.join(characters_list)

In [102]:
print(sample(network, 2000, initilaization_characters_instead_of_random_start='He', top_k=5, cuda=True))

Hed intine this he was and sonesomede as he he thought, and went out of
a thing talked of him he had. To sens warking one was nothing three
of his wands to see to the cansuate at off, his hand, when
he came upon there had been as to a sumber. Alexey Alexandrovitch
came to the stranger, went into the stood.

There was struck of at on his was at the same weased in a lift was
son horses, and the find with his words of the resson to the highes
and standenessess in a great of setther that she was not to a man, and the
mutten of the stor, and that which he saw at the carrage, and without the
crose, and at him, and a somet in she saming out the send a chiler, he he
dreamed the still and his wand, and talked a land the poticuat too of
this handent as shout the prothers, while had now had been dreaming at
their hapse, and he held he had sat the succers with her
eyes, and something with a mindle share, and, she definded that he had
to conversation, thinging in the semotims. He had centeddened in

## Loading a checkpoint

In [103]:
# Here we have loaded in a model that trained over 1 epoch `rnn_1_epoch.net`
with open('lstm_epoch.network', 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = LSTM_Char_Generation(checkpoint['tokens'], no_of_hidden_nodes_in_each_lstm_cell=checkpoint['no_of_hidden_nodes_in_each_lstm_cell'], no_of_stacked_lstm_layers=checkpoint['no_of_stacked_lstm_layers'])

loaded.load_state_dict(checkpoint['state_dict'])


IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [104]:
# Change cuda to True if you are using GPU!
print(sample(loaded, 2000, cuda=True, top_k=5, initilaization_characters_instead_of_random_start="And Levin said"))

And Levin said the talle of his words, she had to be the consideress he saw to herself
the player, there wordd there and who house the chold an hows.
She had bound to the stancons of the some of the cold to his for
the particulars to he could be all at a sholl brooked as side,
answered into the sension of that she consemetted her and with was
hearing, as though of her hat the same of a tarriagion and so the talking to
her eness. And the prince was stood on the felt, was the complaire
which won't be so here with a look with the criding, went that was
had saw the said on the ropes, stepped how to his has and had the sate
wat to the sort of the should he did. "I'm a cease of more of still
and to the more in his word, but see in the sone that when he had not stand
him, and though he had began to that, when she had been done, but his
stall of with a
people conficted he saw him and that some to her ears was a stand
of the collision of this his awful to the tarring, they with when the
conceir

In [ ]:

#-------------------------------___Self------------------------------------
a= np.array( [[[1,2,3],
             [4,5,6]],
             
             [[7,8,9],
             [10,11,12]],
             
             [[13,14,15],
             [16,17,18]],
              
              [[19,20,21],
             [22,23,24]]
            
            
            ])

print(a.shape)

b = a.reshape(8, 3)
print(b)
